<a href="https://colab.research.google.com/github/Owen150/Machine-Learning-Tasks/blob/main/Task_Two_GroupWork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import io
import numpy as np
import csv
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LassoCV
from matplotlib import pyplot as plt
from google.colab import files
uploaded = files.upload()

Saving Housing_Pricing.csv to Housing_Pricing.csv


In [3]:
data = pd.read_csv(io.BytesIO(uploaded['Housing_Pricing.csv']))


In [4]:
print(data)


        Id  MSSubClass MSZoning  LotFrontage  LotArea Street LotShape  \
0        1          60       RL         65.0     8450   Pave      Reg   
1        2          20       RL         80.0     9600   Pave      Reg   
2        3          60       RL         68.0    11250   Pave      IR1   
3        4          70       RL         60.0     9550   Pave      IR1   
4        5          60       RL         84.0    14260   Pave      IR1   
...    ...         ...      ...          ...      ...    ...      ...   
1455  1456          60       RL         62.0     7917   Pave      Reg   
1456  1457          20       RL         85.0    13175   Pave      Reg   
1457  1458          70       RL         66.0     9042   Pave      Reg   
1458  1459          20       RL         68.0     9717   Pave      Reg   
1459  1460          20       RL         75.0     9937   Pave      Reg   

     LandContour Utilities LotConfig  ... EnclosedPorch 3SsnPorch ScreenPorch  \
0            Lvl    AllPub    Inside  ... 

In [5]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [6]:
# We first drop the ID feature since we will not be using it
data.drop('Id', axis=1,inplace=True)
data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000


# **Finding The Missing Values**

In [7]:
#Viewing which columns have missing values and which ones do not have missing values
data.isnull().any()

MSSubClass       False
MSZoning         False
LotFrontage       True
LotArea          False
Street           False
                 ...  
MoSold           False
YrSold           False
SaleType         False
SaleCondition    False
SalePrice        False
Length: 76, dtype: bool

In [8]:
data.shape

(1460, 76)

In [9]:
#Total number of (Original features) columns
#Remember we Removed the Id column
original_features=list(data.columns)
len(original_features)

76

In [10]:
#Finding the exact number of columns(features) with missing values
features_missing_values = list(data.columns[data.isnull().any()])
len(features_missing_values)

15

# **Filling The Missing Values**

In [11]:
bfill, mean_fill= [],[]

In [12]:
#Finding the percentages for features with missing values together with their datatypes
from pandas.core.arrays.sparse import dtype
for feature in features_missing_values:
  print(feature, '', data[feature].isnull().sum()/len(data)*100,
        '',data[feature].dtype)
  if data[feature].dtype =='float64':
    mean_fill.append(feature)
  else:
    bfill.append(feature)

LotFrontage  17.73972602739726  float64
MasVnrType  0.547945205479452  object
MasVnrArea  0.547945205479452  float64
BsmtQual  2.5342465753424657  object
BsmtCond  2.5342465753424657  object
BsmtExposure  2.6027397260273974  object
BsmtFinType1  2.5342465753424657  object
BsmtFinType2  2.6027397260273974  object
Electrical  0.0684931506849315  object
FireplaceQu  47.26027397260274  object
GarageType  5.5479452054794525  object
GarageYrBlt  5.5479452054794525  float64
GarageFinish  5.5479452054794525  object
GarageQual  5.5479452054794525  object
GarageCond  5.5479452054794525  object


In [13]:
#Numerical fields can be filled by mean_fill
mean_fill

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [14]:
#Getting the mean and filling it
for feature in mean_fill:
  data[feature].fillna(data[feature].mean(),inplace=True)

In [15]:
#It fills the bfill using method bfill strategy
for feature in bfill:
  data[feature].fillna(method='bfill',inplace=True)

In [16]:
#Checking whether there is any feature with missing values
data.columns[data.isnull().any()]

Index(['FireplaceQu'], dtype='object')

In [17]:
data['FireplaceQu'].isnull().sum()

2

In [18]:
data['FireplaceQu'].fillna(method='ffill',inplace = True)

In [23]:
#Checking whether there is any feature with missing values
data.isnull().any()

MSSubClass       False
MSZoning         False
LotFrontage      False
LotArea          False
Street           False
                 ...  
MoSold           False
YrSold           False
SaleType         False
SaleCondition    False
SalePrice        False
Length: 76, dtype: bool

In [24]:
#Finding the exact number of columns(features) with missing values
features_missing_values=list(data.columns[data.isna().any()])
len(features_missing_values)

0

# **Encoding The Dataset**

In [25]:
#Selecting object datatypes(one with text) this is done by casting
#Copying to not tamper with the original dataset
cat_data=data.select_dtypes(include=['object']).copy()
cat_data.head(2)

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal
1,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,...,TA,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal


In [26]:
#Form categorical data by grtting the features that need encoding
categorical_features=list(cat_data.columns)
categorical_features

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [27]:
#Nominal one uses one hot encording
#Ordinal (order) data uses categorical encoding
nominal=['MSZoning','LandContour','LotConfig', 'Neighborhood']
ordinal=list(set(categorical_features)-set(nominal))
numerical=list(set(original_features)-set(categorical_features))
target=['SalePrice']

In [28]:
numerical

['OverallQual',
 'BsmtFullBath',
 'Fireplaces',
 'PoolArea',
 'BsmtHalfBath',
 '2ndFlrSF',
 'SalePrice',
 '3SsnPorch',
 'BedroomAbvGr',
 'TotalBsmtSF',
 'MoSold',
 'YearBuilt',
 'GarageYrBlt',
 'OverallCond',
 'TotRmsAbvGrd',
 'EnclosedPorch',
 'HalfBath',
 'MiscVal',
 'WoodDeckSF',
 'YrSold',
 'LowQualFinSF',
 'GarageCars',
 'BsmtFinSF1',
 'OpenPorchSF',
 'MSSubClass',
 'GrLivArea',
 'YearRemodAdd',
 'GarageArea',
 'ScreenPorch',
 'BsmtFinSF2',
 'KitchenAbvGr',
 'FullBath',
 'LotArea',
 '1stFlrSF',
 'MasVnrArea',
 'LotFrontage',
 'BsmtUnfSF']

In [30]:
#Categorical encoding
for feature in ordinal: 
  data[feature]=(data[feature].astype('category')).cat.codes

In [31]:
#Assigning the categorical encoding
df_ordinal=data[ordinal]

In [32]:
#One hot encoding
df_nominal=pd.get_dummies(data[nominal])

In [34]:
#Assigning the one hot encoding
df_numerical=data[numerical]

In [35]:
data[numerical]

,OverallQual,BsmtFullBath,Fireplaces,PoolArea,BsmtHalfBath,2ndFlrSF,SalePrice,3SsnPorch,BedroomAbvGr,TotalBsmtSF,...,GarageArea,ScreenPorch,BsmtFinSF2,KitchenAbvGr,FullBath,LotArea,1stFlrSF,MasVnrArea,LotFrontage,BsmtUnfSF
0,7,1,0,0,0,854,208500,0,3,856,...,548,0,0,1,2,8450,856,196.0,65.0,150
1,6,0,1,0,1,0,181500,0,3,1262,...,460,0,0,1,2,9600,1262,0.0,80.0,284
2,7,1,1,0,0,866,223500,0,3,920,...,608,0,0,1,2,11250,920,162.0,68.0,434
3,7,1,1,0,0,756,140000,0,3,756,...,642,0,0,1,1,9550,961,0.0,60.0,540
4,8,1,1,0,0,1053,250000,0,4,1145,...,836,0,0,1,2,14260,1145,350.0,84.0,490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,6,0,1,0,0,694,175000,0,3,953,...,460,0,0,1,2,7917,953,0.0,62.0,953
1456,6,1,2,0,0,0,210000,0,3,1542,...,500,0,163,1,2,13175,2073,119.0,85.0,589
1457,7,0,2,0,0,1152,266500,0,4,1152,...,252,0,0,1,2,9042,1188,0.0,66.0,877
1458,5,1,0,0,0,0,142125,0,2,1078,...,240,0,1029,1,1,9717,1078,0.0,68.0,0


In [41]:
#Concatinating the encoded dataset
encoded_data=pd.concat([df_numerical, df_nominal, df_ordinal], axis=1)

In [42]:
encoded_data.shape

(1460, 111)

In [43]:
#Viewing 3 rows of the encoded dataset
encoded_data.head(3)

,OverallQual,BsmtFullBath,Fireplaces,PoolArea,BsmtHalfBath,2ndFlrSF,SalePrice,3SsnPorch,BedroomAbvGr,TotalBsmtSF,...,ExterQual,HouseStyle,RoofStyle,BsmtCond,GarageCond,LandSlope,LotShape,FireplaceQu,CentralAir,SaleCondition
0,7,1,0,0,0,854,208500,0,3,856,...,2,5,1,3,4,0,3,4,1,4
1,6,0,1,0,1,0,181500,0,3,1262,...,3,2,1,3,4,0,3,4,1,4
2,7,1,1,0,0,866,223500,0,3,920,...,2,5,1,3,4,0,0,4,1,4


# **Standardizing The Dataset**

In [44]:
#Drop the Sale price in the X axis
X=encoded_data.drop('SalePrice', axis=1)

In [45]:
#Assigning the Saleprice to the Y axis
y=data[target]
y

,SalePrice
0,208500
1,181500
2,223500
3,140000
4,250000
...,...
1455,175000
1456,210000
1457,266500
1458,142125


In [46]:
X.to_numpy()
X

,OverallQual,BsmtFullBath,Fireplaces,PoolArea,BsmtHalfBath,2ndFlrSF,3SsnPorch,BedroomAbvGr,TotalBsmtSF,MoSold,...,ExterQual,HouseStyle,RoofStyle,BsmtCond,GarageCond,LandSlope,LotShape,FireplaceQu,CentralAir,SaleCondition
0,7,1,0,0,0,854,0,3,856,2,...,2,5,1,3,4,0,3,4,1,4
1,6,0,1,0,1,0,0,3,1262,5,...,3,2,1,3,4,0,3,4,1,4
2,7,1,1,0,0,866,0,3,920,9,...,2,5,1,3,4,0,0,4,1,4
3,7,1,1,0,0,756,0,3,756,2,...,3,5,1,1,4,0,0,2,1,0
4,8,1,1,0,0,1053,0,4,1145,12,...,2,5,1,3,4,0,0,4,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,6,0,1,0,0,694,0,3,953,8,...,3,5,1,3,4,0,3,4,1,4
1456,6,1,2,0,0,0,0,3,1542,2,...,3,2,1,3,4,0,3,4,1,4
1457,7,0,2,0,0,1152,0,4,1152,5,...,0,5,1,1,4,0,3,2,1,4
1458,5,1,0,0,0,0,0,2,1078,4,...,3,2,3,3,4,0,3,2,1,4


In [47]:
y=y.to_numpy() #change to a matrix
y

array([[208500],
       [181500],
       [223500],
       ...,
       [266500],
       [142125],
       [147500]])

In [48]:
X.shape #shows all rows and columns

(1460, 110)

In [49]:
y.shape #shows the rows of the Sale price

(1460, 1)

In [50]:
X=StandardScaler().fit_transform(X)
y=StandardScaler().fit_transform(y)
#Normalizing the dataset with Standard scaler

In [51]:
y
#Showing the array of Y

array([[ 0.34727322],
       [ 0.00728832],
       [ 0.53615372],
       ...,
       [ 1.07761115],
       [-0.48852299],
       [-0.42084081]])

In [54]:
X.shape

(1460, 110)

# **Feature Extraction Using PCA**

In [55]:
pca=PCA(n_components=2) #feature extraction

In [56]:
p_components = pca.fit_transform(X)
p_components

array([[ 2.41938473, -1.3522971 ],
       [ 0.4296678 ,  2.597372  ],
       [ 2.83601372, -0.8697221 ],
       ...,
       [ 1.23972735, -0.81292412],
       [-2.8627105 ,  3.17231501],
       [-0.79132442,  2.678853  ]])

In [57]:
pca.explained_variance_ratio_

array([0.10199855, 0.04342156])

# **Feature Selection Using L1**

In [58]:
#new dataset for training
regressor=LassoCV()

In [59]:
#Training the model
regressor.fit(X,y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LassoCV()

In [60]:
regressor.coef_  #seeing the learnt weights

array([ 1.68855827e-01,  3.69060797e-02,  3.33323743e-02, -0.00000000e+00,
        2.95588941e-03,  0.00000000e+00,  6.22109110e-03, -1.29579070e-02,
        3.78020836e-02, -8.47296631e-03,  5.20908134e-02,  0.00000000e+00,
        5.66551730e-02,  4.76953807e-02, -0.00000000e+00,  2.41381869e-03,
       -0.00000000e+00,  2.10645290e-02, -8.56907463e-03, -5.82454788e-04,
        8.59992311e-02,  4.18626496e-02,  9.02563185e-04, -6.16766785e-02,
        2.64125217e-01,  1.64853334e-02,  0.00000000e+00,  3.06470291e-02,
        6.92650022e-03, -1.83295504e-02,  3.52592393e-02,  4.64577948e-02,
        1.14972021e-03,  3.03657765e-02, -2.49552955e-02, -0.00000000e+00,
       -2.00557699e-02,  7.06811206e-03, -0.00000000e+00,  0.00000000e+00,
       -7.69365371e-04, -3.49775886e-02,  1.80467719e-02, -0.00000000e+00,
        0.00000000e+00, -0.00000000e+00,  1.99972639e-02, -1.78014669e-02,
       -1.21327916e-02,  0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        0.00000000e+00,  

In [61]:
regressor.intercept_
#Finding one that intercepts both axis

2.9655600087900183e-16

# **Splitting The Dataset to Train and Test Models**